<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPTモデルを1から実装

In [ ]:
# GPTモデルの設定
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 語彙のサイズ
    "context_length": 1024, # コンテキストの長さ
    "emb_dim": 768,         # 埋め込み次元数
    "n_heads": 12,          # Attentionヘッドの数
    "n_layers": 12,         # 層の数
    "drop_rate": 0.1,       # ドロップアウト率
    "qkv_bias": False       # クエリ、キー、バリューの計算にバイアスを使用するか
}

In [ ]:
# GPTプレイスホルダモデルアーキテクチャ
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # TransformerBlockにはプレースホルダを使う
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        # LayerNormにもプレースホルダを使う(正規化層)
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module):
    # 後ほど本物のTransformerBlockに置き換える
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        # 何もせずに入力を返すだけ
        return x

class DummyLayerNorm(nn.Module):
    # 後ほど本物のLAyerNormに置き換える
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [ ]:
# GPTモデル用のトークナイザーを使用して
# 2つのテキスト入力からなるバッチをトークン化する
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

In [ ]:
# DummyGPTModelインスタンスを生成して
# トークン化したバッチbatchを入力として渡す
# モデルの出力は一般的にロジットと言う
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape: ", logits.shape)
print(logits)

## 4.2 層正規化を使って活性化を正規化する
- 勾配消失、勾配爆発を抑える
- layer nomarizationで層正規化を行い、ニューラルネットワーク層の出力（活性化）を平均0、分散1（単位分散）になるように調節
- 層正規化を行うことで、効果的な重みへの収束が早まり、訓練の一貫性と信頼性が確保される。
- 現代のTransformerアーキテクチャでは、layer normalaizationはMulti-head Attentionモジュールの前または後に適用するのが一般的

In [ ]:
# layer normalaizationの例
torch.manual_seed(123)

# それぞれ5つの次元（特徴量）を持つ2つの訓練サンプルを作成
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

出力結果の1行目は1つ目の入力に対する層の出力を表しており、2行目は2つ目の入力に対する層の出力を表している

In [ ]:
# Layer Normalizationの出力の平均と分散を調べる
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print(f"Mean: {mean}")
print(f"Variance: {var}")

平均や分散の計算時にkeepdim=Trueを使うと、dimで指定された次元に沿ってテンソルを縮小したとしても、出力テンソルの次元数は入力テンソルと同じになる

In [ ]:
# 取得した層の出力にLayer Normalizationを適用
# 出力から平均を引き、分散の平方根（標準偏差）で割る -> 標準化する
out_norm = (out - mean) / torch.sqrt(var)
print(f"Normalization layer outputs:\n {out_norm}")

mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print(f"Mean:\n {mean}")
print(f"Variance:\n {var}")

In [ ]:
# 読みやすさ向上のため、指数表記をオフにすることもできる
torch.set_printoptions(sci_mode=False)
print(f"Mean: \n{mean}")
print(f"Variance: \n{var}")

In [ ]:
# 元に戻す
torch.set_printoptions(sci_mode=True)

In [ ]:
# Layer Normalizationの実装と適用をモジュール化

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False) # unbiased=True 有偏分散
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # 標準化（ゼロ除算にならないようにepsを加える）
        return self.scale * norm_x + self.shift

### 有偏分散
`unbiased=False`を設定することで、実装依存の詳細を利用する

分散計算において、入力の数`n`で除算を行うことになる

ベッセル補正では、標本分散推定でのバイアスを調節するために分母を`n`ではなく`n-1`を使うのが一般的

LLMでは、埋め込みの次元数`n`が非常に大きいため、`n`と`n-1`の差は実質的に無視できる

また、オリジナルのTensorFlowのデフォルトの振る舞いを反映しているからという理由もあり有偏分散を用いる

In [ ]:
# LayerNormモジュールを実際にバッチに適用
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print(f"Mean: \n{mean}")
print(f"Variance: \n{var}")

バッチ正規化 -> バッチ次元に沿って正規化を行う

層正規化（`Layer Normalization`）-> 特徴量の次元に沿って正規化を行う

LLMは膨大な計算リソースが必要になることが多く、層正規化はバッチサイズとは関係なく各入力を正規化するため、そうしたシナリオで柔軟性と安定性が向上する

このため、分散学習を行う場合や、リソースに制約がある環境でモデルをデプロイする場合に、特に層正規化は効果が期待できる。

## 4.2 GELU活性化を使ってフィードフォワードネットワークを実装する
- GELUとSwiGLUの活性化関数が強い
- それぞれガウス線形ユニットとシグモイドゲート線形ユニットを組み込んでいる
- 単純なReLUの活性関数に比べてディープラーニングモデルの性能向上に寄与する